# Experiment with jacobian as a linear operator

In [1]:
import numpy as np
from regressor import LinearRegressor

from inversion_ideas import DataMisfit, TikhonovZero

In [2]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

array([0.78225148, 0.67148671, 0.2373809 , 0.17946133, 0.34662367,
       0.15210999, 0.31142952, 0.23900652, 0.54355731, 0.91770851])

In [3]:
# Build the X array
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [4]:
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
noise = rng.normal(scale=1e-2 * maxabs, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

array([2.83840696, 2.18091081, 2.00623242, 2.08333039, 2.01694883,
       2.7826232 , 2.10564027, 1.27333506, 2.08859855, 1.94177648,
       1.88492037, 2.92394733, 2.17231952, 3.08009275, 1.61670886,
       1.77403753, 2.67305005, 1.91413882, 2.42117827, 2.13991628,
       2.0153805 , 2.71388471, 2.65944255, 2.44416121, 3.14217523])

## Linear regressor with jacobian as a dense matrix

Define the simulation

In [5]:
simulation = LinearRegressor(X, linop=False)

And a random model

In [6]:
model = np.random.default_rng(seed=404).uniform(size=simulation.n_params)

Evaluate the jacobian

In [7]:
simulation.jacobian(model)  # should return a dense array

array([[4.44264723e-01, 7.60284086e-01, 5.75280767e-01, 5.11884120e-01,
        6.57160266e-01, 9.46730040e-01, 9.15303691e-01, 7.20234663e-01,
        1.22754477e-01, 9.09341434e-01],
       [7.13282058e-01, 3.53302920e-01, 2.21366873e-01, 9.14078303e-01,
        8.15687052e-01, 3.40313380e-01, 9.62818089e-01, 6.83787068e-01,
        3.63987604e-02, 3.84888353e-01],
       [1.42086599e-01, 3.54204403e-01, 9.67718984e-01, 5.95587095e-01,
        3.95789476e-01, 1.83878615e-01, 2.77425050e-01, 7.18244514e-01,
        9.17009668e-01, 4.22921723e-01],
       [1.83523965e-01, 5.00748020e-01, 8.23541669e-01, 3.63411724e-01,
        4.88826840e-01, 5.71664781e-01, 8.73267349e-01, 6.96450781e-01,
        8.27504216e-01, 2.34782839e-01],
       [3.60832015e-01, 7.87289140e-03, 9.92841209e-01, 1.37940229e-01,
        7.17729471e-01, 7.83053514e-01, 8.17056956e-01, 2.37562075e-01,
        2.96404717e-01, 6.44919192e-01],
       [2.46095918e-01, 7.12840133e-01, 2.62346116e-01, 3.63872300e-01,
   

Define a data misfit

In [8]:
uncertainty = 1e-2 * maxabs * np.ones_like(synthetic_data)
data_misfit = DataMisfit(synthetic_data, uncertainty, simulation)

The gradient of the data misfit is a vector

In [9]:
data_misfit.gradient(model)

array([-18149.85993886, -18257.25261202, -15886.68171464, -14251.52010139,
       -16918.68409749, -16925.96156027, -18263.07429758, -15436.48921498,
       -20292.70015609, -23260.24022026])

The hessian is a linear operator because the data misfit has `build_hessian=False`

In [10]:
data_misfit.hessian(model)

<10x10 _ProductLinearOperator with dtype=float64>

Define an objective function with regularization

In [11]:
smallness = TikhonovZero(n_params)
phi = data_misfit + 1e-3 * smallness
phi

φd(m) + 0.00 φs(m)

The hessian of `phi` should also be a dense array

In [12]:
phi.hessian(model)

<10x10 _SumLinearOperator with dtype=float64>

## Linear regressor with jacobian as a linear operator

Define the simulation

In [13]:
simulation = LinearRegressor(X, linop=True)

Evaluate the jacobian

In [14]:
simulation.jacobian(model)  # should return a linear operator

<25x10 _CustomLinearOperator with dtype=float64>

In [15]:
assert np.allclose(simulation(model), simulation.jacobian(model) @ model)  # true because the simulation is linear

Define a data misfit

In [16]:
uncertainty = 1e-2 * maxabs * np.ones_like(synthetic_data)
data_misfit = DataMisfit(synthetic_data, uncertainty, simulation)

The gradient of the data misfit is a vector

In [17]:
data_misfit.gradient(model)

array([-18149.85993886, -18257.25261202, -15886.68171464, -14251.52010139,
       -16918.68409749, -16925.96156027, -18263.07429758, -15436.48921498,
       -20292.70015609, -23260.24022026])

The hessian here is still a linear operator, both because `build_hessian=False` and because the jacobian is now a linear operator as well.

In [18]:
data_misfit.hessian(model)

<10x10 _ProductLinearOperator with dtype=float64>

Define an objective function with regularization

In [19]:
smallness = TikhonovZero(n_params)
phi = data_misfit + 1e-3 * smallness
phi

φd(m) + 0.00 φs(m)

The hessian of `phi` should also be a linear operator

In [20]:
phi.hessian(model)

<10x10 _SumLinearOperator with dtype=float64>